<a href="https://colab.research.google.com/github/monash-tp37-project/iteration_1_model/blob/master/bert_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and use pretrained BERT model for news classification

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Installing the Hugging Face PyTorch Interface for Bert
# !pip install pytorch-pretrained-bert pytorch-nlp
!pip install -q transformers
!pip3 install keybert
!pip3 install keyphrase-vectorizers

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 49.0 MB/s 
     |████████████████████████████████| 6.6 MB 41.8 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 232 kB 11.9 MB/s 
     |████████████████████████████████| 51 kB 6.3 MB/s 
     |████████████████████████████████| 1.2 MB 67.2 MB/s 
  Created wheel for keybert: filename=keybert-0.5.1-py3-none-any.whl size=21332 sha256=2a8926385d8431c65ceb4e5818b24444c8b35b3dc802f1cafd60f3109286bc65
  Stored in directory: /root/.cache/pip/wheels/8e/95/c5/f5ceed2a9f9e80bc1a706a10a6fb03d726df7a3dd11800a58b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=896b7c30d8c4decef5e79e8dc79d1381d449a4d6e983b9dfe06a7e8a331c15cd
  Stored in directory: /root/.cache/pip/wheel

In [3]:
import os
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

# libraries for keywords
from keybert import KeyBERT
import keyphrase_vectorizers
from keyphrase_vectorizers import KeyphraseCountVectorizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
current_dir = os.getcwd()
current_dir

'/content'

In [5]:
#import torch
#from transformers import AutoTokenizer, AutoModelForSequenceClassification

# read in trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained(f"{current_dir}/drive/MyDrive/colab_data/tokenizer_150k_512_10k_fake")
model = BertForSequenceClassification.from_pretrained(f"{current_dir}/drive/MyDrive/colab_data/model_150k_512_16epoch_10k_fake")

In [10]:
# Remove stopwords and remove words with 2 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in english_stop_words:
            result.append(token)
    joined_results = ' '.join(result)
           
    return joined_results

In [11]:
# save list of all stopwords
english_stop_words = stopwords.words('english')

In [31]:
sequences = [preprocess("Trump was selling weapons to Russia so that his company could become rich of the proceeds.")]

# tokenize text input
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

In [32]:
with torch.no_grad():
  # make predictions
  output = model(**tokens)

  # using a softmax activation function get discrete predictions
  predictions = F.softmax(output.logits, dim=1)
  labels = int(torch.argmax(predictions, dim=1))

  # using a sigmoid function get continous predictions 
  label = torch.sigmoid(predictions).tolist()[0]
  reliable = round(label[0],2)
  unreliable = round(label[1],2)
  print(f'Your news has a {unreliable}% probability of being unreliable and a {reliable}% of being reliable.')

  if reliable < unreliable:

    print('Therefore, our model predicts your news is likely to be unreliable.')

  else:

    print('Therefore, our model predicts your news is likely to be reliable.')

Your news has a 0.73% probability of being unreliable and a 0.5% of being reliable.
Therefore, our model predicts your news is likely to be unreliable.


In [ ]:
kw_model = KeyBERT()

In [ ]:
keywords = kw_model.extract_keywords(docs=sequences, vectorizer=KeyphraseCountVectorizer())

/usr/local/lib/python3.7/dist-packages/keybert/_model.py:131: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  "Although extracting keywords for multiple documents is faster "
1it [00:00, 424.48it/s]


In [ ]:
[x[0] for x in sorted(keywords[0], key=lambda x: x[1], reverse=True)][0:3]

['elbert county clerk dallas schroeder',
 'sensitive voting information',
 'county']